In [1]:
import numpy as np
import pandas as pd
from astropy.io import fits
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, Flatten, Reshape, Conv2D, MaxPooling2D, AveragePooling2D, UpSampling2D

### [data](http://archive.stsci.edu/), [table of labelled data properties](http://exoplanetarchive.ipac.caltech.edu/)

In [4]:
# load data
koi = pd.read_csv('../KOI.csv')
sysid = 6022556
obsq1 = fits.open(f'../data/{sysid}/kplr00{sysid}-2009166043257_llc.fits')

# get time and flux data
spec_cols = np.array((obsq1[1].data.TIME-min(obsq1[1].data.TIME),obsq1[1].data.PDCSAP_FLUX)).transpose()
df = pd.DataFrame(spec_cols, columns=['time','flux'])
df.head()

,time,flux
0,0.000000,8591.976562
1,0.020434,8587.601562
2,0.040869,8595.933594
3,0.061303,8586.216797
4,0.081738,8587.034180


### [vision transformer](https://towardsdatascience.com/a-demonstration-of-using-vision-transformers-in-pytorch-mnist-handwritten-digit-recognition-407eafbc15b0)

In [1]:
N_EPOCHS = 25
train_data = df.time
test_data = df.flux

model = ViT(image_size=1, patch_size=7, num_classes=10, channels=1,dim=64, depth=6, heads=8, mlp_dim=128)
optimizer = optim.Adam(model.parameters(), lr=0.003)

train_loss_history, test_loss_history = [], []
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_epoch(model, optimizer, train_data, train_loss_history)
    evaluate(model, test_data, test_loss_history)

NameError: name 'df' is not defined

[pypi tutorial](https://pypi.org/project/vision-transformer-pytorch/)

In [10]:
from vision_transformer_pytorch import VisionTransformer

# load an efficient net
model = VisionTransformer.from_name('ViT-B_16')

# load pretrained model
model_p = VisionTransformer.from_pretrained('ViT-B_16')

In [11]:
!pip -q install vit_pytorch linformer

[image example](https://github.com/lucidrains/vit-pytorch/blob/main/examples/cats_and_dogs.ipynb)

## Trying a regular schmegular CNN

In [2]:
kepler = pd.read_csv("../KOI.csv")
kepler.head()

,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,10811496,K00753.01,NaN,CANDIDATE,CANDIDATE,0.000,0,0,0,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [3]:
# columns of interest
cols = ['koi_period','koi_time0bk','koi_impact','koi_duration','koi_depth','koi_prad','koi_teq','koi_insol','koi_model_snr',
        'koi_steff','koi_slogg','koi_srad','koi_pdisposition']

# get specified columns and drop rows with NaN values
k = kepler[cols].dropna(axis=0)

# change classification values to 0 (false positive) and 1 (candidate)
k['koi_pdisposition'].mask(k['koi_pdisposition'] == 'CANDIDATE', 1, inplace=True)
k['koi_pdisposition'].mask(k['koi_pdisposition'] == 'FALSE POSITIVE', 0, inplace=True)
k.head()

,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,koi_pdisposition
0,9.488036,170.538750,0.146,2.95750,615.8,2.26,793.0,93.59,35.8,5455.0,4.467,0.927,1
1,54.418383,162.513840,0.586,4.50700,874.8,2.83,443.0,9.11,25.8,5455.0,4.467,0.927,1
2,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638.0,39.30,76.3,5853.0,4.544,0.868,1
3,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395.0,891.96,505.6,5805.0,4.564,0.791,0
4,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406.0,926.16,40.9,6031.0,4.438,1.046,1


In [4]:
# create numpy arrays for the input and target data
input_data = k[cols].drop(columns=['koi_pdisposition']).values
target_data = k['koi_pdisposition'].values.astype('int')

# set number of inputs and outputs
num_inputs, num_outputs = len(input_data[0]), 1

# split data into train and test parts
xtrain, xtest, ytrain, ytest = train_test_split(input_data, target_data, test_size = 0.3)
print(xtrain.shape,xtest.shape,ytrain.shape,ytest.shape)

(6440, 12) (2761, 12) (6440,) (2761,)


In [5]:
# super basic keras neural network
inputlayer = Input(shape=(num_inputs,))
x = Dense(50, activation = 'relu')(inputlayer)
x = Dense(50, activation = 'relu')(x)
outputlayer = Dense(2, activation = 'softmax')(x)
model = Model(inputs=inputlayer, outputs=outputlayer)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
dense (Dense)                (None, 50)                650       
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 102       
Total params: 3,302
Trainable params: 3,302
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(input_data, target_data, epochs = 100, batch_size = 36, validation_split = 0.2, shuffle = True, verbose = 0)
model.evaluate(input_data, target_data, batch_size = 36)

256/256 [==============================] - 0s 1ms/step - loss: 1.3895 - accuracy: 0.7373


[1.3895055055618286, 0.7373111844062805]

In [ ]:
# sample convolution model for images

inputlayer = Input(shape=(32,12))
conv1 = Conv2D(filters=5, kernel_size = 4, activation = 'relu')(inputlayer)
pool1 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(conv1)

conv2 = Conv2D(filters=8, kernel_size = 4, activation = 'relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(conv2)

flatten = Flatten(data_format=None)(pool2)
outputlayer = Dense(10, activation = 'softmax')(flatten)                   
model = Model(inputs=inputlayer, outputs=outputlayer)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(xtrain, ytrain, epochs = 10, batch_size = 32, validation_split = 0.2, shuffle = True, verbose = 0)